In [1]:
!pip install selenium
!pip install webdriver_manager

### Aula extra - Como analisar os dados de rentabilidade?

### Desafio
    * Construir um código  que faça um ranking dos melhores fundos em três periodos de rentabilidade diferentes. Iremos selecionar is 10 melhores posicionados no quesito rentabilidade no curto e no longo prazo.

### Passo a passo
Passo 1 - Puxar as tabelas igual na aula 2 <br>
Passo 2 - Escolher quais periodos de rentabilidade iremos analisar. <br>
Passo 3 - Retirar os dados faltantes. Fundos que não possuem os periodos necessarios, ficarão de fora. <br>
Passo 4 - Transformar as rentabilidades em números decimais, ao invés de string.<br>
Passo 5 - Juntar as tabelas, igual na aula 2. <br>
Passo 6 - Filtrar os ETFs alavancados. <br>
Passo 7 - Fazer os rankings de rentabilidade por periodos e soma-los. <br>
Passo 8 - Ordenar do menor ao maior raning, encontrado os "melhores" ETFs. <br>



## Passo 1 - Puxar as tabelas da aula 2

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Instalando o driver
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

# Abrindo a pagina desejada
driver.get('https://www.etf.com/etfanalytics/etf-finder')

# tempo para carregar o js
time.sleep(10)

# mudando a pagina para 100 elementos
botao_100 = driver.find_element("xpath",'/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span')

#botao_100.click()

## se der algum problema de click
driver.execute_script("arguments[0].click();", botao_100)

time.sleep(2)
numero_paginas = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/label[2]')
numero_paginas = numero_paginas.text.replace("of", "")
numero_paginas = int(numero_paginas)
numero_paginas


##
lista_tabela_por_pagina = []

elemento = driver.find_element("xpath",'/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/div' )


for pagina in range(1, numero_paginas + 1):
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
    
    botao_proximo = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/span[2]')
    
    #botao_proximo.click()
    driver.execute_script("arguments[0].click();", botao_100)
    
    

tabela_cadastro_etfs = pd.concat(lista_tabela_por_pagina)
#tabela_cadastro_etfs


### voltano as paginas 
formulario_voltar_pagina = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/input')
formulario_voltar_pagina.clear()
formulario_voltar_pagina.send_keys('1')
formulario_voltar_pagina.send_keys(u'\ue007')

##
clicando_perfomace = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/ul/li[2]/span')
clicando_perfomace.click()


## A partir daqui é tudo igual

lista_tabela_por_pagina = []

elemento = driver.find_element("xpath",'/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/div' )


for pagina in range(1, numero_paginas + 1):
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
    
    botao_proximo = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/span[2]')
    
    
    ## tivesso um bug no clicar igual o de antes
    driver.execute_script("arguments[0].click();", botao_proximo)
    #botao_proximo.click()
    
    

tabela_rentabilidade_etfs = pd.concat(lista_tabela_por_pagina)

driver.quit()



In [3]:
tabela_cadastro_etfs = tabela_cadastro_etfs.set_index('Ticker')
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs.set_index('Ticker')

In [4]:
tabela_rentabilidade_etfs

,Name,1 Month,3 Month,YTD,1 Year,3 Years,5 Years,10 Years,As Of Date
Ticker,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,1.50%,6.90%,7.27%,-9.01%,19.91%,11.05%,12.19%,04/04/23
IVV,iShares Core S&P 500 ETF,1.49%,6.91%,7.27%,-9.02%,19.97%,11.06%,12.24%,04/04/23
VOO,Vanguard S&P 500 ETF,1.53%,6.94%,7.29%,-9.02%,19.98%,11.07%,12.24%,04/04/23
VTI,Vanguard Total Stock Market ETF,0.25%,6.24%,6.75%,-10.30%,19.93%,10.25%,11.74%,04/04/23
QQQ,Invesco QQQ Trust,6.63%,20.25%,20.01%,-12.94%,21.03%,15.65%,17.73%,04/04/23
...,...,...,...,...,...,...,...,...,...
CVLC,Calvert US Large-Cap Core Responsible Index ETF,0.41%,--,--,--,--,--,--,04/04/23
CVMC,Calvert US Mid-Cap Core Responsible Index ETF,-5.55%,--,--,--,--,--,--,04/04/23
CVIE,Calvert International Responsible Index ETF,1.81%,--,--,--,--,--,--,04/04/23


## Passo 2 : Escolher quais periodos de rentabilidade iremos analisar.
1 ano, 3 anos , 5 anos

In [5]:
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs[['1 Year', '3 Years', '5 Years']]
tabela_rentabilidade_etfs

,1 Year,3 Years,5 Years
Ticker,,,
SPY,-9.01%,19.91%,11.05%
IVV,-9.02%,19.97%,11.06%
VOO,-9.02%,19.98%,11.07%
VTI,-10.30%,19.93%,10.25%
QQQ,-12.94%,21.03%,15.65%
...,...,...,...
CVLC,--,--,--
CVMC,--,--,--
CVIE,--,--,--


# Passo 3 -  Retirar os dados faltantes. Fundos que não possuem os periodos necessarios, Ficarao fora

In [6]:
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs.replace('--', pd.NA)

In [7]:
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs.dropna()
tabela_rentabilidade_etfs

,1 Year,3 Years,5 Years
Ticker,,,
SPY,-9.01%,19.91%,11.05%
IVV,-9.02%,19.97%,11.06%
VOO,-9.02%,19.98%,11.07%
VTI,-10.30%,19.93%,10.25%
QQQ,-12.94%,21.03%,15.65%
...,...,...,...
LD,-9.42%,10.26%,-2.12%
ERUS,0%,-33.19%,-22.74%
LTL,-26.30%,12.48%,1.36%


## Passo 4 - Transformar as rentabilidades em números decimais.

In [8]:
for coluna in tabela_rentabilidade_etfs.columns:
    
    tabela_rentabilidade_etfs[coluna] = tabela_rentabilidade_etfs[coluna].str.rstrip('%').astype(float)/100

In [9]:
tabela_rentabilidade_etfs

,1 Year,3 Years,5 Years
Ticker,,,
SPY,-0.0901,0.1991,0.1105
IVV,-0.0902,0.1997,0.1106
VOO,-0.0902,0.1998,0.1107
VTI,-0.1030,0.1993,0.1025
QQQ,-0.1294,0.2103,0.1565
...,...,...,...
LD,-0.0942,0.1026,-0.0212
ERUS,0.0000,-0.3319,-0.2274
LTL,-0.2630,0.1248,0.0136


### Juntar as tabelas as tabelas

In [15]:

## juntando as tabelas numa só
base_final = tabela_cadastro_etfs.join(tabela_rentabilidade_etfs, how='inner')
base_final.head()

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,3 Years,5 Years
Ticker,,,,,,,,
ACWI,iShares MSCI ACWI ETF,Equity: Global - Total Market,Blackrock,0.32%,$18.67B,-0.0834,0.1702,0.0706
ACWI,iShares MSCI ACWI ETF,Equity: Global - Total Market,Blackrock,0.32%,$18.67B,-0.0834,0.1702,0.0706
ACWI,iShares MSCI ACWI ETF,Equity: Global - Total Market,Blackrock,0.32%,$18.67B,-0.0834,0.1702,0.0706
ACWI,iShares MSCI ACWI ETF,Equity: Global - Total Market,Blackrock,0.32%,$18.67B,-0.0834,0.1702,0.0706
ACWI,iShares MSCI ACWI ETF,Equity: Global - Total Market,Blackrock,0.32%,$18.67B,-0.0834,0.1702,0.0706


## Passo 6: Filtrar os etfs alavancados

In [14]:

base_final = base_de_dados_final[~base_de_dados_final['Segment'].str.contains('Leveraged')]


### Passo 7: Fazer os rankings de rentabilididade por periodo e soma-los

In [20]:
## Criandp mais uma coluna
base_final['rank_1_anos'] = base_final['1 Year'].rank(ascending= False)
base_final['rank_3_anos'] = base_final['3 Years'].rank(ascending= False)
base_final['rank_5_anos'] = base_final['5 Years'].rank(ascending= False)


# Fazendo a soma de todas as novas colunas

base_final['rank_final'] = (base_final['rank_1_anos']+ base_final['rank_3_anos']+ base_final['rank_5_anos'])
base_final

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,3 Years,5 Years,rank_1_anos,rank_3_anos,rank_5_anos,rank_final
Ticker,,,,,,,,,,,,
ACWI,iShares MSCI ACWI ETF,Equity: Global - Total Market,Blackrock,0.32%,$18.67B,-0.0834,0.1702,0.0706,1936.5,1616.5,1520.5,5073.5
ACWI,iShares MSCI ACWI ETF,Equity: Global - Total Market,Blackrock,0.32%,$18.67B,-0.0834,0.1702,0.0706,1936.5,1616.5,1520.5,5073.5
ACWI,iShares MSCI ACWI ETF,Equity: Global - Total Market,Blackrock,0.32%,$18.67B,-0.0834,0.1702,0.0706,1936.5,1616.5,1520.5,5073.5
ACWI,iShares MSCI ACWI ETF,Equity: Global - Total Market,Blackrock,0.32%,$18.67B,-0.0834,0.1702,0.0706,1936.5,1616.5,1520.5,5073.5
ACWI,iShares MSCI ACWI ETF,Equity: Global - Total Market,Blackrock,0.32%,$18.67B,-0.0834,0.1702,0.0706,1936.5,1616.5,1520.5,5073.5
...,...,...,...,...,...,...,...,...,...,...,...,...
XLY,Consumer Discretionary Select Sector SPDR Fund,Equity: U.S. Consumer Discretionary,State Street Global Advisors,0.10%,$14.34B,-0.2110,0.1792,0.0894,3056.5,1424.5,1072.5,5553.5
XLY,Consumer Discretionary Select Sector SPDR Fund,Equity: U.S. Consumer Discretionary,State Street Global Advisors,0.10%,$14.34B,-0.2110,0.1792,0.0894,3056.5,1424.5,1072.5,5553.5
XLY,Consumer Discretionary Select Sector SPDR Fund,Equity: U.S. Consumer Discretionary,State Street Global Advisors,0.10%,$14.34B,-0.2110,0.1792,0.0894,3056.5,1424.5,1072.5,5553.5


### Passo 8: Gran finale - Ordenar do menor ranking pro maior, encontrando os melhores ETFs

In [23]:
base_final = base_final.sort_values(by = 'rank_final')

base_final.head(20)

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,3 Years,5 Years,rank_1_anos,rank_3_anos,rank_5_anos,rank_final
Ticker,,,,,,,,,,,,
XLE,Energy Select Sector SPDR Fund,Equity: U.S. Energy,State Street Global Advisors,0.10%,$40.36B,0.1489,0.4855,0.1016,16.5,16.5,816.5,849.5
XLE,Energy Select Sector SPDR Fund,Equity: U.S. Energy,State Street Global Advisors,0.10%,$40.36B,0.1489,0.4855,0.1016,16.5,16.5,816.5,849.5
XLE,Energy Select Sector SPDR Fund,Equity: U.S. Energy,State Street Global Advisors,0.10%,$40.36B,0.1489,0.4855,0.1016,16.5,16.5,816.5,849.5
XLE,Energy Select Sector SPDR Fund,Equity: U.S. Energy,State Street Global Advisors,0.10%,$40.36B,0.1489,0.4855,0.1016,16.5,16.5,816.5,849.5
XLE,Energy Select Sector SPDR Fund,Equity: U.S. Energy,State Street Global Advisors,0.10%,$40.36B,0.1489,0.4855,0.1016,16.5,16.5,816.5,849.5
XLE,Energy Select Sector SPDR Fund,Equity: U.S. Energy,State Street Global Advisors,0.10%,$40.36B,0.1489,0.4855,0.1016,16.5,16.5,816.5,849.5
XLE,Energy Select Sector SPDR Fund,Equity: U.S. Energy,State Street Global Advisors,0.10%,$40.36B,0.1489,0.4855,0.1016,16.5,16.5,816.5,849.5
XLE,Energy Select Sector SPDR Fund,Equity: U.S. Energy,State Street Global Advisors,0.10%,$40.36B,0.1489,0.4855,0.1016,16.5,16.5,816.5,849.5
XLE,Energy Select Sector SPDR Fund,Equity: U.S. Energy,State Street Global Advisors,0.10%,$40.36B,0.1489,0.4855,0.1016,16.5,16.5,816.5,849.5
